In [1]:
import cv2
import pandas as pd
import cvlib as cv
import os 
from pathlib import Path
from utils.util import ensure_dir, prepare_device
from PIL import Image
import tqdm
import shutil

from utils.util import ensure_dir


def crop_coord(imagePath):
	image = cv2.imread(imagePath)
	face, confidence = cv.detect_face(image)
	if len(face)==0 :
		return 0
	x, y, w, h = face[0]
	coord = [x, y, w, h]

	return coord


def crop_face(coord_list, path, crop_path):
	if type(coord_list)==int:
		shutil.copy(path, crop_path)
		return
	x, y, w, h = coord_list
	image = cv2.imread(path)
	H, W, C = image.shape
	image_array = image[max(y-100,0):min(h+100,H), max(0,x-100):min(w+100,W)]
	cv2.imwrite(crop_path, image_array)

In [5]:
# train_with_label.csv에 추가 정보
label_path = '/opt/ml/image-classification-level1-12/templates/data/train/train_with_label.csv'
df = pd.read_csv(label_path)
df['normal'] = df['path'].map(lambda x: 1 if 'normal' in x else 0)
# crop_coor, 얼굴 좌표 반환
df['crop_coor'] = df.apply(lambda x: crop_coord(x['path']) if x['normal']==1 else None, axis=1)
df = df.sort_values(by='name').fillna(method='backfill', limit=6).sort_index()
# crop_image 저장할 경로 설정
df['crop_path'] = df['path'].apply(lambda x: x.replace('images', 'image_crop'))
# crop_image 생성
df.apply(lambda x: crop_face(x['crop_coor'], x['path'], x['crop_path']), axis=1)
# train_with_label에 crop_coord, crop_path 추가해서 재생성
df.to_csv(label_path, index=False)

In [28]:
# Evalution Data에도 crop 적용
# image_crop 추가
eval_path = '/opt/ml/image-classification-level1-12/templates/data/eval'
eval_df = pd.read_csv(eval_path+'/info.csv')
eval_df['crop_coord'] = eval_df.apply(lambda x: crop_coord(eval_path+'/images/'+x['ImageID']),axis=1)

path = eval_path+'/images/'
crop_path = eval_path+'/crop_image/'

eval_df.apply(lambda x : crop_face(x['crop_coord'], path+x['ImageID'], crop_path+x['ImageID']), axis=1)

In [73]:
# train_with_crop.csv 생성
import pandas as pd

df = pd.read_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_label.csv')
df_new = df[['crop_path', 'label']].copy()
df_new.columns = ['path', 'label']
# df_new = pd.DataFrame(df['path'].append(df['crop_path']), columns=['path'])
# df_new['label'] = df['label'].append(df['label'])
df_new['name'] = df_new['path'].apply(lambda x: x.split('/')[-2]+'_'+x.split('/')[-1])
df_new.to_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_crop.csv', index=False)

In [16]:
# Original+Crop image folder
root_folder = '/opt/ml/image-classification-level1-12/templates/data/train/image_crop'
root_dir = [dir for dir in os.listdir(root_folder) if not dir.startswith('._')]
for folder_list in root_dir:
	crop_folder = os.path.join(root_folder, folder_list)
	move_folder = crop_folder.replace('image_crop', 'ori_crop')
	crop_dir = [dir for dir in os.listdir(crop_folder) if not dir.startswith('._')]	
	for image_list in crop_dir:
		shutil.copy(os.path.join(crop_folder, image_list), os.path.join(move_folder, 'crop'+image_list))

In [56]:
df_label = pd.read_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_label.csv')
df_crop = pd.read_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_crop.csv')

label_series = df_label['path'].map(lambda x: x.replace('images', 'ori_crop'))
crop_series = df_crop['path'].map(lambda x: x.replace('image_crop', 'ori_crop'))
crop_series = crop_series.map(lambda x: "/".join(x.split('/')[:-1])+"/crop"+x.split('/')[-1])

df_merge = pd.DataFrame(label_series.append(crop_series), columns=['path'])
df_merge['label'] = df_label['label'].append(df_label['label'])
df_merge = df_merge.reset_index(drop=True)
df_merge['name'] = df_merge['path'].map(lambda x: x.split('/')[-2]+'_'+x.split('/')[-1])
df_merge.to_csv('/opt/ml/image-classification-level1-12/templates/data/train/train_with_all.csv', index=False)